<a href="https://colab.research.google.com/github/DevanshChoubey/rat-sql/blob/master/RatSql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import os 
os.chdir("/content/rat-sql")

In [ ]:
!git clone https://github.com/microsoft/rat-sql

In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.12-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [ ]:
!python --version
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Python 3.7.1


In [ ]:
import os
os.chdir("/content/rat-sql")
!mkdir -p third_party
!git clone https://github.com/salesforce/WikiSQL third_party/wikisql
!gdown https://drive.google.com/uc?id=109s_s_2FWZ9uGBkNc5gPSo2cIF5WW7IS
!mkdir data
!tar -xvf spider.tar.gz -C data
#!curl https://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05.zip | jar xv
!cp -r /content/rat-sql/third_party/wikisql/lib /content/rat-sql

In [ ]:
!cp -r /content/rat-sql/third_party/wikisql/lib /content/rat-sql

In [ ]:
#!pip install jsonnet torchtext transformers babel records asdl astor attrs bpemb networkx pyrsistent nltk stanford-corenlp entmax
import nltk
nltk.download('stopwords')
#!export PYTHONPATH=third_party/wikisql/:$PYTHONPATH
#!export $CORENLP_HOME=/content/rat-sql/third_party/stanford-corenlp-full-2018-10-05

#export PYTHONPATH=/root/rat-sql/third_party/wikisql/:$PYTHONPATH

In [ ]:
!pip install jsonnet~=0.14.0 transformers babel records asdl astor attrs bpemb networkx pyrsistent nltk stanford-corenlp~=3.9.2
!pip install entmax
import nltk
nltk.download('stopwords')
nltk.download('punkt')
!pip install torchtext==0.7

In [ ]:
!mkdir logdir
!mkdir logdir/bs=3,lr=7.4e-04,bert_lr=3.0e-06,end_lr=0e0,att=1

In [ ]:
import time
while True:
  time.sleep(300)
  print("seep")

In [ ]:
!python run.py train experiments/spider-bert-run.jsonnet

In [ ]:
import json
res_json = json.load(open("/content/rat-sql/logdir/bs=3,lr=7.4e-04,bert_lr=3.0e-06,end_lr=0e0,att=1/ie_dirs/bert_run_true_1-step63100.eval"))
print( res_json['total_scores']['all']['exact'])


In [ ]:
!python run.py eval experiments/spider-bert-run.jsonnet 

WARNING <class 'ratsql.models.enc_dec.EncDecModel.Preproc'>: superfluous {'name': 'EncDec'}
WARNING <class 'ratsql.models.enc_dec.EncDecModel'>: superfluous {'decoder_preproc': {'grammar': {'clause_order': None, 'end_with_from': True, 'factorize_sketch': 2, 'include_literals': False, 'infer_from_conditions': True, 'name': 'spider', 'output_from': True, 'use_table_pointer': True}, 'save_path': 'data/spider/nl2code,output_from=true,fs=2,emb=bert,cvlink', 'use_seq_elem_rules': True}, 'encoder_preproc': {'bert_version': 'bert-large-uncased-whole-word-masking', 'compute_cv_link': True, 'compute_sc_link': True, 'db_path': 'data/spider/database', 'fix_issue_16_primary_keys': True, 'include_table_name_in_column': False, 'save_path': 'data/spider/nl2code,output_from=true,fs=2,emb=bert,cvlink'}}
Loading model from logdir/bert_run/bs=6,lr=7.4e-04,bert_lr=3.0e-06,end_lr=0e0,att=1/model_checkpoint-00035300
DB connections: 100% 166/166 [00:13<00:00, 12.76it/s]
100% 1034/1034 [04:02<00:00,  4.26it/s]

In [ ]:
import json
import os
import _jsonnet
from ratsql.commands.infer import Inferer
from ratsql.datasets.spider import SpiderItem
from ratsql.utils import registry

exp_config = json.loads(_jsonnet.evaluate_file("/content/rat-sql/experiments/spider-bert-run.jsonnet"))
model_config_path = os.path.join(root_dir, exp_config["model_config"])
print(model_config_path)
model_config_args = exp_config.get("model_config_args")
infer_config = json.loads(_jsonnet.evaluate_file("/content/rat-sql/experiments/spider-bert-run.jsonnet", tla_codes={'args': json.dumps(model_config_args)})

inferer = Inferer(infer_config)
inferer.device = torch.device("cuda")
model = inferer.load_model(model_dir, checkpoint_step)
dataset = registry.construct('dataset', inferer.config['data']['val'])

for _, schema in dataset.schemas.items():
    model.preproc.enc_preproc._preprocess_schema(schema)

def question(q, db_id):
    spider_schema = dataset.schemas[db_id]
    data_item = SpiderItem(
        text=None,  # intentionally None -- should be ignored when the tokenizer is set correctly
        code=None,
        schema=spider_schema,
        orig_schema=spider_schema.orig,
        orig={"question": q}
    )
    model.preproc.clear_items()
    enc_input = model.preproc.enc_preproc.preprocess_item(data_item, None)
    preproc_data = enc_input, None
    with torch.no_grad():
        return inferer._infer_one(model, data_item, preproc_data, beam_size=1, use_heuristic=True)
